Butona her basıldığında kart üzerindeki LEDler sırasıyla yanacak. Bu işlem kesme (interrupt) ile yapılacak.

main fonksiyonu şu şekilde:  
Öncelikle her zaman olduğu gibi buton için A portuna, LEDler için D portuna clk sinyali veriyoruz. Ayrıca LEDleri output modunda ayarlıyoruz.

In [ ]:
//A ve D portlari etkinlestirildi
RCC->AHB1ENR |= RCC_AHB1ENR_GPIOAEN;
RCC->AHB1ENR |= RCC_AHB1ENR_GPIODEN;

//D portunun 12,13,14,15. pinleri output olarak ayarlandi.
GPIOD->MODER |= (GPIO_MODER_MODE12_0 | GPIO_MODER_MODE13_0
| GPIO_MODER_MODE14_0 | GPIO_MODER_MODE15_0);

interrrupları A portunun 0. pininden alacağımız için EXTI0 kesmesini etkinleştirmek gerekli. Bu kesme vektör tablosunda 6 numaralı kesme olduğu için NVIC_EnableIRQ(6); şeklinde etkinleştirme yapılabilir. Ama header dosyasında enumeration yapısıyla EXTI0_IRQn ifadesine 6 sayısı atanmış. Dlayısıyla aktifleştirme işlemi şu şekilde yapılıyor:  

In [ ]:
//NIVIC te 6 nolu interrupt etkinlesitirildi.
//NVIC_EnableIRQ(6);
NVIC_EnableIRQ(EXTI0_IRQn);

sonraki adımda ilgili pin için interrupt maskesi etkinleştiriliyor: 

In [ ]:
// A portunun 0. pinine bagli buton icin interrupt maski etkinlestirildi.
EXTI->IMR |= EXTI_IMR_IM0;

Kesme işleminin sinyalin yükselen kenarında veya düşen kenarında olmasını ayarlamak gerekiyor.

In [ ]:
//A portunun 0. pinine bagli butona basildiginda 0'dan 1'e gecme durumunda interrup olacak.
EXTI->RTSR |= EXTI_RTSR_TR0;

Sonra programı sonsuz boş bir döngüde bırakıyoruz:

In [ ]:
while(1);

main fonksiyonunun dışında butona her basıldığını saymak için gloabal bir değişken tanımlanıyor:

In [ ]:
unsigned long int LED_i=0;

İnterrupt olduğunda cihazın nasıl davranacağını belirlemiş olduğumuz interrupt handler fonksiyonu şu şekilde tanımlanmalıdır:

In [ ]:
void EXTI0_IRQHandler(void){

}

Yukarıdaki tanımlama projedeki assembly dosyasında belirlenmiştir.   
İnterrupt handler fonksiyonuna eklenmesi gereken kodlar şu şekildedir:

Öncelikle pending register kontrol edilir ve ilgili bite 1 yazılarak interruptla ilgili işlemler yapılmaya başlanır. Bu işlemin interruptun başında yapıması gerekmektedir.

In [ ]:
//pending register control
if(EXTI->PR & EXTI_PR_PR0)
    EXTI->PR |= EXTI_PR_PR0;

daha sonracihazınızın interrupt işleminde nasıl çalışmasını istiyorsak ona göre programlamamızı yaparız. 

In [ ]:
LED_i++;

//All LEDs are OFF
GPIOD->ODR &= ~(GPIO_ODR_OD12 | GPIO_ODR_OD13 | GPIO_ODR_OD14 | GPIO_ODR_OD15);    
// LED i ON        
GPIOD->ODR |= 1U << (12+(LED_i%4));

Yapılan tüm kodlama şu şekildedir:

In [ ]:
#include "stm32f4xx.h"                  // Device header

unsigned long int LED_i=0;

void EXTI0_IRQHandler(void){
    
    //pending register control
    if(EXTI->PR & EXTI_PR_PR0)
        EXTI->PR |= EXTI_PR_PR0;
    
    LED_i++;
    
    //All LEDs are OFF
    GPIOD->ODR &= ~(GPIO_ODR_OD12 | GPIO_ODR_OD13 | GPIO_ODR_OD14 | GPIO_ODR_OD15);    
    // LED i ON        
    GPIOD->ODR |= 1U << (12+(LED_i%4));
    
}

int main(){
    
    //A ve D portlari etkinlestirildi
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIOAEN;
    RCC->AHB1ENR |= RCC_AHB1ENR_GPIODEN;
    
    //D portunun 12,13,14,15. pinleri output olarak ayarlandi.
    GPIOD->MODER |= (GPIO_MODER_MODE12_0 | GPIO_MODER_MODE13_0 | GPIO_MODER_MODE14_0 | GPIO_MODER_MODE15_0);

    //NIVIC te 6 nolu interrupt etkinlesitirildi.
    //NVIC_EnableIRQ(6);
    NVIC_EnableIRQ(EXTI0_IRQn);
    
    // A portunun 0. pinine bagli buton icin interrupt maski etkinlestirildi.
    EXTI->IMR |= EXTI_IMR_IM0; 
    
    //A portunun 0. pinine bagli butona basildiginda 0'dan 1'e gecme durumunda interrup olacak.
    EXTI->RTSR |= EXTI_RTSR_TR0;
    
    while(1);
    
    return 1;
}
